# Generador de personas aleatorias

In [1]:
import random as rm
import numpy as np
import pandas as pd
from faker import Faker
import datetime as dt
import pytz
import re
from geopy.geocoders import Nominatim
from timezonefinder import TimezoneFinder
from funciones import quitar_acentos
import urllib.parse
from typing import Union

In [17]:
def sacar_curp(nombre: str, apellido_paterno: str, apellido_materno: str, sexo: str, fecha_nac: Union[str, dt.datetime], estado_nac: str) -> str:
	nombre = quitar_acentos(nombre).upper().replace('Ñ', 'X')
	apellido_paterno = quitar_acentos(apellido_paterno).upper().replace('Ñ', 'X')
	apellido_materno = quitar_acentos(apellido_materno).upper().replace('Ñ', 'X')
	estado_nac = quitar_acentos(estado_nac.replace(' ', '_').lower())
	if isinstance(fecha_nac, str): fecha_nac = dt.datetime.strptime(fecha_nac, '%Y-%m-%d')
	# Primera letra y vocal del primer apellido
	curp = apellido_paterno[0:2].upper()
	# Primera letra del segundo apellido
	curp += apellido_materno[0:1].upper()
	# Primera letra del nombre de pila
	curp += nombre[0:1].upper()
	# Fecha de nacimiento (2 últimos dígitos del año, 2 del mes y 2 del día de nacimiento)
	curp += fecha_nac.strftime('%y%m%d')
	# Letra del sexo (H o M)
	curp += sexo[0:1].upper()
	# Dos letras correspondientes a la entidad de nacimiento segun RENAPO (Sonora => SR)
	# en el caso de extranjeros, se marca como NE (Nacido Extranjero)
	clave_estados = {
		'aguascalientes': 'AS', 'baja_california': 'BC', 'baja_california_sur': 'BS',
		'campeche': 'CC', 'coahuila': 'CL', 'colima': 'CM', 'chiapas': 'CS', 'chihuahua': 'CH',
		'distrito_federal': 'DF', 'ciudad_de_mexico': 'DF', 'durango': 'DG', 'guanajuato': 'GT',
		'guerrero': 'GR', 'hidalgo': 'HG', 'jalisco': 'JC', 'mexico': 'MC', 'estado_de_mexico': 'MC',
		'michoacan': 'MN', 'morelos': 'MS', 'nayarit': 'NT', 'nuevo_leon': 'NL', 'oaxaca': 'OC',
		'puebla': 'PL', 'queretaro': 'QT', 'quintana_roo': 'QR', 'san_luis_potosi': 'SP', 'sinaloa': 'SL',
		'sonora': 'SR', 'tabasco': 'TC', 'tamaulipas': 'TS', 'tlaxcala': 'TL', 'veracruz': 'VZ',
		'yucatan': 'YN', 'zacatecas': 'ZS'
	}
	if estado_nac in clave_estados:
		clave_estado_nac = clave_estados[estado_nac]
	else:
		clave_estado_nac = 'NE'
	curp += clave_estado_nac
	# Primera consonante interna del primer apellido
	curp += re.sub(f'[aeiou]', '', apellido_paterno)[1:2].upper()
	# Primera consonante interna del segundo apellido
	curp += re.sub(f'[aeiou]', '', apellido_materno)[1:2].upper()
	# Primera consonante interna del nombre
	curp += re.sub(f'[aeiou]', '', nombre)[1:2].upper()
	# Dígito verificador del 0-9 para fechas de nacimiento hasta el año 1999 y A-Z para fechas de nacimiento a partir del 2000
	int_char = [rm.randint(0, 9), chr(rm.randint(65, 90))]
	curp += str(int_char[0] if fecha_nac.year <= 1999 else int_char[1])
	# Homoclave, para evitar duplicaciones
	curp += str(rm.randint(0, 9))
	return curp

def obtener_coordenadas(ciudad, estado, pais = 'Mexico'):
	try:
		# Hacer peticion a un api para obtener coordenadas del centro y extremas
		geolocator = Nominatim(user_agent = 'geoapiExercises')
		location = geolocator.geocode(f'{ciudad} {estado} {pais}')
		# En caso de que no se haya encontrado nada
		if not location: raise BaseException(f'No se encontraron coordenadas de "{ciudad}", "{estado}", "{pais}"')
		location = location.raw
		# Obtener coordenadas del centro
		centro = (float(location['lat']), float(location['lon']))
		# Obtener extremos originales
		minx, maxx, miny, maxy = [float(i) for i in location['boundingbox']]
		ne, no, se, so = [(maxx, maxy), (maxx, miny), (minx, maxy), (minx, miny)]
		print(f'\033[33mobtener_coordenadas()\033[0m: Coordenadas encontradas en "{ciudad}", "{estado}", "{pais}"')
	except BaseException as e:
		print(f'\033[31mobtener_coordenadas()\033[0m:', e)
		return None
	return {'centro': centro, 'noreste': ne, 'noroeste': no, 'sureste': se, 'suroeste': so}

def generar_puntos_random(centro, extremo_so, extremo_ne, n = 1, reducir = True):
	# Obtener coordenadas del centro
	cenx, ceny = centro
	# Obtener extremos originales
	minx, miny = extremo_so
	maxx, maxy = extremo_ne
	# Hallar punto medio entre (suroeste centro) y (centro noreste)
	minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
	maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# True para reducir el area de la ciudad
	if reducir:
		# Hallar punto medio de puntos medios y centro
		minx, miny = ((minx + cenx) / 2), ((miny + ceny) / 2)
		maxx, maxy = ((cenx + maxx) / 2), ((ceny + maxy) / 2)
	# Obtener n puntos aleatorios dentro del area
	puntos_random = []
	while len(puntos_random) < n:
		puntos_random.append((rm.uniform(minx, maxx), rm.uniform(miny, maxy)))
	return puntos_random

def buscar_direccion(puntos_random):
	geolocator = Nominatim(user_agent = 'geoapiExercises')
	dirrr = {}
	listo = False
	for punto_random in puntos_random:
		dirr = {}
		address = geolocator.reverse(punto_random, language = 'es_MX')
		direccion_raw = address.raw
		direccion_raw = direccion_raw['address']
		print(direccion_raw)
		# {
		#   'house_number': '71',
		#   'quarter': 'Zapotitos',
		#   'hamlet': 'San José Dos',
		#   'village': 'San Felipe',
		#   'city': 'villa insurgentes  (El Calabazal)',
		# }
		# {
		#   "building": "Jardin De Niños Gabino Barrera",
		#   "road": "Avenida Azarco",
		#   "residential": "Villa Insurgentes (El Calabazal)",
		#   "suburb": "Barrio El Cerrito",
		#   "town": "villa insurgentes  (El Calabazal)",
		#   "county": "Sombrerete",
		#   "state": "Zacatecas",
		#   "postcode": "99109",
		#   "country": "México",
		#   "country_code": "mx"
		# }
		# {
		#   'amenity': 'Colegio de Estudios Científicos y Tecnológicos del Estado de Sonora',
		#   'house_number': 's/n',
		#   'road': 'Calle Periférico',
		#   'town': 'Bacobampo',
		#   'county': 'Etchojoa',
		#   'state': 'Sonora',
		#   'postcode': '85287',
		#   'country': 'México',
		#   'country_code': 'mx'
		# }
		"""
		dirr['numero'] = direccion_raw['house_number'] if 'house_number' in direccion_raw else rm.randint(1, 999)
		dirr['cp'] = direccion_raw['postcode']
		dirr['ciudad'] = direccion_raw['city']
		dirr['estado'] = direccion_raw['state']
		dirr['pais'] = direccion_raw['country']
		"""
		direccion = address[0].split(', ')
		print(direccion)
		count = len(direccion)
		print('\033[34m', count, '\033[0m')
		if count <= 2: continue
		if count == 10:
			print('\033[34mHay de 10\033[0m')
			print(list(direccion))
		elif count == 9:
			print('\033[34mHay de 9\033[0m')
			print(list(direccion))
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 8:
			dirr['local'] = direccion[0]
			if direccion[1].isnumeric(): dirr['numero'] = direccion[1]
			else: dirr['numero'] = rm.randint(1, 999)
			dirr['calle'] = direccion[2]
			dirr['colonia'] = direccion[3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 7:
			dirr['local'] = direccion[0]
			dirr['calle'] = direccion[1]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[2]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 6:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['ciudad'] = direccion[-4]
			dirr['pais'] = direccion[-1]
			listo = True
		elif count == 5:
			dirr['calle'] = direccion[0]
			dirr['numero'] = rm.randint(1, 999)
			dirr['colonia'] = direccion[1]
			dirr['ciudad'] = direccion[-3]
			if direccion[-2].isnumeric():
				dirr['cp'] = direccion[-2]
				dirr['estado'] = direccion[-3]
			else:
				dirr['cp'] = str(rm.randint(80000, 99999))
				dirr['estado'] = direccion[-2]
			dirr['pais'] = direccion[-1]
			listo = True
		else:
			continue
		if listo:
			dirr['arr_original'] = direccion
			print('\033[33mbuscar_direccion()\033[0m: Direccion encontrada en', punto_random)
			dirrr = dirr
			break
	if not listo:
		print('\033[33mbuscar_direccion()\033[0m: No se encontro una direccion')
	return dirrr

In [18]:
reap = pd.read_csv('./conjunto_datos/relacion_edad_altura_peso.csv')
nombres = pd.read_csv('./catalogos/nombres.csv')
profesiones = pd.read_csv('./catalogos/profesiones.csv', header = None)
file_ciudades = './catalogos/ciudades.csv'
ct_ciudades = pd.read_csv(file_ciudades)

def personas_aleatorias(n = 5):
	fake = Faker(['es_MX'])
	personas = []
	for _ in range(n):
		print('Idx', _)
		sexo = rm.choice(['H', 'M'])
		# generar tipo de sangre y asignar probabilidad a cada valor para que salga
		tipo_sangre = rm.choices(['A+', 'A-', 'B+', 'B-', 'AB+', 'AB-', 'O+', 'O-'], weights = [30, 6, 9, 2, 4, 1, 39, 9])
		tipo_sangre = tipo_sangre[0]

		fecha_nac = fake.date_of_birth(minimum_age = int(reap['edad'].min()))
		# calcular edad en base a la fecha de nacimiento
		dia_actual = dt.date.today()
		edad = dia_actual.year - fecha_nac.year - ((dia_actual.month, dia_actual.day) < (fecha_nac.month, fecha_nac.day))

		# dependiendo de la edad se busca el margen de alturas y pesos que tenga esa edad
		# si es mayor que la edad maxima que este registrada se le da el margen de esa edad maxima
		# mucha gente deja de crecer a los 20
		margen_ap = reap[(reap['edad'] == edad) if edad <= reap['edad'].max() else (reap['edad'] == reap['edad'].max())].to_dict(orient = 'records')[
			0]

		# dependiendo del sexo se generan nombres femeninos o masculinos
		# y estaturas y pesos dentro de su margen correspondiente
		if sexo == 'H':
			titulo = rm.choice(['Sr.', 'Dr.', 'Mtro.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['h'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.h.cm'], margen_ap['lim_sup.altura.h.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.h.kg'], margen_ap['lim_sup.peso.h.kg']), 2)
		else:
			titulo = rm.choice(['Sra.', 'Dra.', 'Mtra.', 'Lic.', 'Ing.']) if edad >= 18 and rm.randint(0, 1) else None
			nombre = rm.choice(nombres['m'])
			altura = round(rm.uniform(margen_ap['lim_inf.altura.m.cm'], margen_ap['lim_sup.altura.m.cm']), 2)
			peso = round(rm.uniform(margen_ap['lim_inf.peso.m.kg'], margen_ap['lim_sup.peso.m.kg']), 2)

		apellido_paterno = fake.last_name()
		apellido_materno = fake.last_name()

		id_estado = rm.choice(ct_ciudades['id_estado'].unique())
		ct_ciudad = ct_ciudades[ct_ciudades['id_estado'] == id_estado].sample()
		ct_ciudad_index_row = ct_ciudad.index.values[0]
		ct_ciudad = ct_ciudad.to_dict(orient = 'records')[0]

		ciudad = ct_ciudad['ciudad']
		estado = ct_ciudad['estado']
		curp = sacar_curp(nombre, apellido_paterno, apellido_materno, sexo, fecha_nac, estado)
		profesion = profesiones.sample().values[0][0] if edad >= 18 else None
		tel = f'({ct_ciudad["lada"]})' + ' ' + '{:03d}'.format(rm.randint(0, 999)) + ' ' + '{:04d}'.format(rm.randint(0, 9999))

		# Generar un nombre de usuario para un email
		switcher = {
			1: fake.user_name(),
			# aperez
			2: f'{nombre[0]}{apellido_paterno}',
			# alonso_perez
			3: f'{nombre}_{apellido_paterno}',
			# aps70
			4: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.strftime("%y")}',
			# aps1970
			5: f'{nombre[0]}{apellido_paterno[0]}{apellido_materno[0]}{fecha_nac.year}',
			# alonsoperezsoltero
			6: f'{nombre}{apellido_paterno}{apellido_materno}'
		}
		rand_int = rm.randint(min(list(switcher.keys())), max(list(switcher.keys())))
		usuario = switcher.get(rand_int, 'usuario')
		# quitar espacios o reemplazarlos por un _
		usuario = usuario.replace(' ', rm.choice(['', '_'])).lower()
		usuario = quitar_acentos(usuario)
		subdominio = rm.choice(['.com', '.net', '.io', '.gob', '.org', '.edu'])
		correo = usuario + f'@example{subdominio}'

		"""
		El Número de Seguridad Social (NSS) es una homoclave que asigna el IMSS
		para llevar un registro de los trabajadores y asegurados
		que están inscritos en dicha institución.
		El NSS es único, permanente e intransferible.
		"""
		# El NSS está compuesto por 11 dígitos:
		nss = None
		if profesion:
			# Los primeros dos caracteres están vinculados a la subdelegación en el que la persona fue afiliada.
			nss1 = ('%02d' % rm.randint(0, 99))
			# Los dos dígitos siguientes indican el año en el que la persona se afilió al Seguro Social.
			nss2 = ('%02d' % rm.randint(0, 99))
			# Los siguientes dos dígitos corresponden a la fecha de nacimiento del afiliado.
			nss3 = fecha_nac.strftime('%y')
			# Los cuatro números siguientes son los dígitos que asigna el IMSS al trabajador.
			nss4 = ('%02d' % rm.randint(0, 9999))
			# El último dígito corresponde al número de verificación del trabajador en el IMSS.
			nss5 = str(rm.randint(0, 9))
			nss = f'{nss1} {nss2} {nss3} {nss4} {nss5}'

		"""Coordenadas de la ciudad"""
		centro = suroeste = noreste = None
		# si el catalogo ya tiene coordenadas, tomar esos valores
		if pd.notna(ct_ciudad['centro']) and pd.notna(ct_ciudad['suroeste']) and pd.notna(ct_ciudad['noreste']):
			centro = eval(ct_ciudad['centro'])
			suroeste = eval(ct_ciudad['suroeste'])
			noreste = eval(ct_ciudad['noreste'])
		else:
			# si no, buscar en internet el punto central y extremos segun la ciudad
			try:
				coordenadas = obtener_coordenadas(ciudad, estado)
				if coordenadas:
					centro = coordenadas['centro']
					suroeste = coordenadas['suroeste']
					noreste = coordenadas['noreste']
					# guardar coordenadas en el catalogo
					# para que a la otra no haya que buscar en internet
					ct_ciudades.loc[ct_ciudad_index_row, 'centro'] = str(centro)
					ct_ciudades.loc[ct_ciudad_index_row, 'suroeste'] = str(suroeste)
					ct_ciudades.loc[ct_ciudad_index_row, 'noreste'] = str(noreste)
					ct_ciudades.to_csv(file_ciudades, index = False)
			except BaseException as be:
				# En caso de no haber encontrado o no tener internet
				print(f'\033[31mCoordsException: idx {_}\033[0m')
				print(be)

		"""Direccion"""
		direcc = cp = None
		if centro and suroeste and noreste:
			puntos_random = generar_puntos_random(centro, suroeste, noreste, 5)
			try:
				direccion = buscar_direccion(puntos_random)
				if direccion and all(k in direccion for k in ('calle', 'numero', 'colonia', 'cp')):
					calle = direccion['calle']
					numero = direccion['numero']
					colonia = direccion['colonia']
					cp = direccion['cp']
					direcc = f'{calle} #{numero}, {colonia}'
			except BaseException as be:
				print(f'\033[31mAddressException: idx {_}\033[0m')
				print(be)

		"""Zona horaria"""
		timezone = tz_offset = None
		if centro:
			# obtener zona horaria segun latitud y longitud
			obj = TimezoneFinder()
			#if centro[1] > 180.0 or centro[1] < -180.0 or centro[0] > 90.0 or centro[0] < -90.0:
			timezone = obj.timezone_at(lat = centro[0], lng = centro[1])
			# en Enero no hay DST
			#tz_offset = pytz.timezone(timezone).localize(dt.datetime(2011, 1, 1)).strftime('%z')
			# 'datetime.now' tiene DST (Daylight Saving Time)
			tz_offset = dt.datetime.now(pytz.timezone(timezone)).strftime('%z')

		if not centro: centro = (np.nan, np.nan)

		personas.append({
			'titulo': titulo,
			'nombre': nombre,
			'apellido_paterno': apellido_paterno,
			'apellido_materno': apellido_materno,
			'fecha_nac': fecha_nac.strftime('%Y-%m-%d'),
			'sexo': sexo,
			'altura': altura,
			'peso': peso,
			'tipo_sangre': tipo_sangre,
			'curp': curp,
			'correo': correo,
			'tel': tel,
			'color_fav': fake.safe_color_name(),
			'profesion': profesion,
			'nss': nss,
			'direccion': direcc,
			'cp': cp,
			'ciudad': ciudad,
			'estado': estado,
			'lat': centro[0],
			'lon': centro[1],
			'zona_horaria': timezone,
			'tz_offset': tz_offset,
		})
	return pd.DataFrame(personas)

personas_aleatorias(10)

Idx 0
obtener_coordenadas(): Coordenadas encontradas en "Tizimín", "Yucatan", "Mexico"
{'road': 'Calle 65', 'city': 'Tizimin', 'county': 'Tizimín', 'state': 'Yucatán', 'postcode': '97775', 'country': 'México', 'country_code': 'mx'}
['Calle 65', 'Tizimin', 'Tizimín', 'Yucatán', '97775', 'México']
 6 
buscar_direccion(): Direccion encontrada en (21.13570572930175, -88.15170310333221)
Idx 1
obtener_coordenadas(): Coordenadas encontradas en "Cadereyta", "Nuevo Leon", "Mexico"
{'road': 'Cadereyta', 'residential': 'Sierra Ventana', 'city': 'Monterrey', 'county': 'Monterrey', 'state': 'Nuevo León', 'postcode': '647770', 'country': 'México', 'country_code': 'mx'}
['Cadereyta', 'Sierra Ventana', 'Monterrey', 'Nuevo León', '647770', 'México']
 6 
buscar_direccion(): Direccion encontrada en (25.63150818798151, -100.28575397476868)
Idx 2
obtener_coordenadas(): No se encontraron coordenadas de "Santa Maria Nativitas", "Tlaxcala", "Mexico"
Idx 3
{'road': 'Calle Río Mayo', 'village': 'Agua Blanca', '

,titulo,nombre,apellido_paterno,apellido_materno,fecha_nac,sexo,altura,peso,tipo_sangre,curp,...,profesion,nss,direccion,cp,ciudad,estado,lat,lon,zona_horaria,tz_offset
0,None,Victoria,Reséndez,Huerta,2018-11-20,M,92.07,16.91,A+,REHV181120MYNEUIG0,...,None,None,"Calle 65 #54, Tizimin",97775,Tizimín,Yucatan,21.142892,-88.152472,America/Merida,-0600
1,Lic.,Pilar,Olmos,Tovar,1946-02-12,H,169.47,85.30,O+,OLTP460212HNLLOI44,...,Director de Marketing,18 59 46 3550 3,"Cadereyta #203, Sierra Ventana",647770,Cadereyta,Nuevo Leon,25.631512,-100.285784,America/Monterrey,-0600
2,Sr.,Espartaco,Estrada,Salas,1946-08-12,H,170.69,83.63,A+,ESSE460812HTLSAS17,...,Tripulante de cabina,82 64 46 407 3,None,None,Santa Maria Nativitas,Tlaxcala,NaN,NaN,None,None
3,Sra.,Marisol,Tejeda,Carrera,1951-01-27,M,161.21,58.32,AB+,TECM510127MSREAA85,...,Tecnólogo de ropa/textil,41 86 51 6696 5,"Calle Río Mayo #864, Agua Blanca",85290,Agua Blanca,Sonora,27.114295,-109.732980,America/Hermosillo,-0700
4,None,Flavio,Zelaya,Durán,1954-11-10,H,167.52,63.33,O+,ZEDF541110HNTEUL43,...,Consejero,35 49 54 6739 2,"El Tizate #469, Santiago Ixcuintla",63569,El Tizate,Nayarit,21.802413,-105.115132,America/Mazatlan,-0700
5,Lic.,Noemí,Sosa,Duarte,1926-06-08,M,165.35,71.23,B+,SODN260608MZSOUO52,...,Científico biomédico,30 80 26 2772 9,"Aveninda Ferrocarril #216, San Jerónimo",98610,San Jerónimo,Zacatecas,22.652976,-102.491953,America/Mexico_City,-0600
6,Ing.,Ivonne,Duarte,Reynoso,1935-09-11,M,173.83,53.74,O+,DURI350911MBSUEV54,...,Jefe de estado mayor,47 91 35 3848 4,"Calle Gandhi #689, San José del Cabo",23407,San José del Cabo,Baja California Sur,23.059836,-109.702515,America/Mazatlan,-0700
7,None,Natalia,Mesa,Godoy,1946-01-03,M,165.38,81.98,B+,MEGN460103MCSEOA37,...,Tesorero corporativo,87 01 46 3744 4,"Calle Tercera Oriente Norte #534, San Pedro Bu...",96405,San Pedro Buenavista,Chiapas,16.135230,-93.165035,America/Mexico_City,-0600
8,None,Berta,Rosas,Páez,1907-03-03,M,169.94,70.30,O+,ROPB070303MMNOAE31,...,Comprador minorista,12 07 07 2149 2,"Calle Benito Juárez #455, Lombardía",86295,Gabriel Zamora (Lombardia),Michoacan,19.157562,-102.058639,America/Mexico_City,-0600
9,None,Clemente,Blanco,Montañez,2018-04-02,H,91.03,15.58,O+,BLMC180402HSRLOLJ1,...,None,None,"Calle Periférico #181, Bacobampo",85287,Bacobampo,Sonora,26.982309,-109.653611,America/Hermosillo,-0700
